In [13]:
import numpy as np
import pandas as pd
import requests

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

In [14]:
def get_ipo_data(target):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    }

    url = "https://stockanalysis.com/ipos/" + target + "/"

    # header is not necessary here
    response = requests.get(url, headers=headers)

    ipo_raw = pd.read_html(response.text)
    
    return ipo_raw[0]

In [15]:
df_ipo_filings = get_ipo_data('filings')
df_ipo_filings.head(20)

C:\Users\Sean\AppData\Local\Temp\ipykernel_24016\4258465984.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_raw = pd.read_html(response.text)


,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"May 3, 2024",TBN,Tamboran Resources Corporation,-,-
1,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000
2,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
3,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
4,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-
5,"Apr 22, 2024",DRJT,Derun Group Inc,$5.00,-
6,"Apr 19, 2024",GPAT,GP-Act III Acquisition Corp.,$10.00,25000000
7,"Apr 16, 2024",JLJT,Jialiang Holdings Ltd,$5.00,-
8,"Apr 15, 2024",GAUZ,Gauzy Ltd.,-,-
9,"Apr 12, 2024",BOW,Bowhead Specialty Holdings Inc.,-,-


In [16]:
df_ipo_filings.dtypes

Filing Date       object
Symbol            object
Company Name      object
Price Range       object
Shares Offered    object
dtype: object

# Q1

In [17]:
df_ipo_filings['Filing Date'] = pd.to_datetime(df_ipo_filings['Filing Date'], format="%b %d, %Y")
df_ipo_filings['Shares Offered'] = pd.to_numeric(df_ipo_filings['Shares Offered'], errors='coerce').fillna(0)

In [18]:
def average_price_range(row):
    if row == "-":
        return np.NaN
    elif '-' in row:
        prices = row.replace('$','').split(' - ')
        prices = [float(item) for item in prices]

        return sum(prices)/len(prices)
    elif '$' in row:
        return float(row.replace('$',''))
    else:
        return row

In [19]:
df_ipo_filings

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,2024-05-03,TBN,Tamboran Resources Corporation,-,0.0
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,0.0
...,...,...,...,...,...
320,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000.0
321,2020-01-21,UTXO,"UTXO Acquisition, Inc.",$10.00,5000000.0
322,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000.0
323,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000.0


In [20]:
df_ipo_filings['avg_price'] = df_ipo_filings['Price Range'].apply(average_price_range).fillna(0)
df_ipo_filings

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,avg_price
0,2024-05-03,TBN,Tamboran Resources Corporation,-,0.0,0.00
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,3.00
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,10.00
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.00
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,0.0,0.00
...,...,...,...,...,...,...
320,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000.0,9.00
321,2020-01-21,UTXO,"UTXO Acquisition, Inc.",$10.00,5000000.0,10.00
322,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000.0,9.00
323,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000.0,4.00


In [21]:
df_ipo_filings['shares_offered_value'] = df_ipo_filings['Shares Offered'] * df_ipo_filings['avg_price']
df_ipo_filings

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,avg_price,shares_offered_value
0,2024-05-03,TBN,Tamboran Resources Corporation,-,0.0,0.00,0.0
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,3.00,11250000.0
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,10.00,60000000.0
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.00,50000000.0
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,0.0,0.00,0.0
...,...,...,...,...,...,...,...
320,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000.0,9.00,13500000.0
321,2020-01-21,UTXO,"UTXO Acquisition, Inc.",$10.00,5000000.0,10.00,50000000.0
322,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000.0,9.00,22500000.0
323,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000.0,4.00,17200000.0


In [22]:
df_ipo_fri_2023 = df_ipo_filings[(df_ipo_filings['Filing Date'].dt.dayofweek == 4) & (df_ipo_filings['Filing Date'].dt.year == 2023)]
q1_ans = df_ipo_fri_2023.groupby('Filing Date').agg(totalsum=('shares_offered_value', 'sum')).reset_index()

In [23]:
q1_ans['totalsum_M'] = round(q1_ans['totalsum']/1e6)
q1_ans.totalsum_M.sum()

285.0

# Q2

In [24]:
df_ipo_2023 = get_ipo_data("2023")
df_ipo_2024 = get_ipo_data("2024")

C:\Users\Sean\AppData\Local\Temp\ipykernel_24016\4258465984.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_raw = pd.read_html(response.text)
C:\Users\Sean\AppData\Local\Temp\ipykernel_24016\4258465984.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_raw = pd.read_html(response.text)


In [25]:
df_ipo_2024.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"May 1, 2024",VIK,Viking Holdings Ltd.,$24.00,$28.94,20.58%
1,"Apr 26, 2024",ZONE,"CleanCore Solutions, Inc.",$4.00,$3.18,-20.63%
2,"Apr 25, 2024",RBRK,"Rubrik, Inc.",$32.00,$34.64,8.97%
3,"Apr 25, 2024",LOAR,Loar Holdings Inc.,$28.00,$51.10,82.50%
4,"Apr 25, 2024",MRX,Marex Group plc,$19.00,$19.15,0.79%


In [26]:
df_ipo = pd.concat([df_ipo_2023, df_ipo_2024], axis=0)
df_ipo = df_ipo[df_ipo['Symbol'] != 'RYZB']

In [27]:
df_ipo[df_ipo['Symbol'] == 'IROH']

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Dec 27, 2023",IROH,Iron Horse Acquisitions Corp.,$10.00,$10.05,0.50%


In [28]:
ticket_list = df_ipo['Symbol'].to_list()

['IROH',
 'LGCB',
 'ZKH',
 'BAYA',
 'INHD',
 'AFJK',
 'GSIW',
 'FEBO',
 'CLBR',
 'ELAB',
 'RR',
 'DDC',
 'SHIM',
 'GLAC',
 'SGN',
 'HG',
 'CRGX',
 'ANSC',
 'AITR',
 'GVH',
 'LXEO',
 'PAPL',
 'ATGL',
 'MNR',
 'WBUY',
 'NCL',
 'BIRK',
 'GMM',
 'PMEC',
 'LRHC',
 'GPAK',
 'SPKL',
 'QETA',
 'MSS',
 'ANL',
 'SYRA',
 'VSME',
 'LRE',
 'TURB',
 'MDBH',
 'KVYO',
 'CART',
 'DTCK',
 'NMRA',
 'ARM',
 'SPPL',
 'NWGL',
 'SWIN',
 'IVP',
 'NNAG',
 'SRM',
 'SPGC',
 'LQR',
 'NRXS',
 'FTEL',
 'MIRA',
 'PXDT',
 'CTNT',
 'HRYU',
 'SRFM',
 'PRZO',
 'HYAC',
 'KVAC',
 'JNVR',
 'ELWS',
 'WRNT',
 'TSBX',
 'ODD',
 'APGE',
 'NETD',
 'SGMT',
 'BOWN',
 'SXTP',
 'PWM',
 'VTMX',
 'INTS',
 'SVV',
 'KGS',
 'FIHL',
 'GENK',
 'BUJA',
 'BOF',
 'AZTR',
 'CAVA',
 'ESHA',
 'ATMU',
 'ATS',
 'IPXX',
 'CWD',
 'SGE',
 'SLRN',
 'ALCY',
 'KVUE',
 'GODN',
 'TRNR',
 'AACT',
 'JYD',
 'USGO',
 'UCAR',
 'WLGS',
 'TPET',
 'TCJH',
 'GDTC',
 'VCIG',
 'GDHG',
 'ARBB',
 'ISPR',
 'MGIH',
 'MWG',
 'HSHP',
 'SFWL',
 'SYT',
 'HKIT',
 'CHSN',
 'T

In [29]:
df_iroh = yf.download(tickers='IROH', period='max', interval='1d')

[*********************100%%**********************]  1 of 1 completed


In [30]:
def download_ipo_data(ticket_list):
    dataframes = {}

    for i in ticket_list:
        df = yf.download(tickers=i, period='max', interval='1d')

        if not df.empty:
            dataframes[f'df_{i}'] = df
        else:
            print(f'No data found for {i}')

    return dataframes

In [31]:
stock_data_dfs = download_ipo_data(ticket_list)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

No data found for PTHR


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

No data found for BKHA


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [93]:
df_test = stock_data_dfs['df_AESI']

In [69]:
(stock_data_dfs['df_AESI']['Adj Close'] / stock_data_dfs['df_AESI']['Adj Close'].shift(2))

Date
2023-03-09         NaN
2023-03-10         NaN
2023-03-13    0.943363
2023-03-14    1.006667
2023-03-15    1.003752
                ...   
2024-05-01    0.942110
2024-05-02    0.987843
2024-05-03    1.000000
2024-05-06    1.044667
2024-05-07    1.015931
Name: Adj Close, Length: 293, dtype: float64

In [104]:
temp_result = pd.DataFrame({'symbol':['df_AESI']})
# Calculate growth rates
for i in range(1, 3):
    # Ensure that there is an element at index i after the shift
    if i < len(df_test):  # Check if 'i' is a valid index in the DataFrame after the shift
        growth_value = (df_test['Adj Close'] / df_test['Adj Close'].shift(i)).iloc[i]
        temp = pd.DataFrame({
            'symbol': 'df_AESI',
            f'growth_future_' + str(i) + 'd': [growth_value]
        })  # Using a consistent index [0] for easy concatenation
        temp_result = pd.merge(temp_result, temp, how='left', on='symbol')
        

    symbol  growth_future_1d
0  df_AESI          0.973451
    symbol  growth_future_2d
0  df_AESI          0.943363


In [105]:
temp_result

,symbol,growth_future_1d,growth_future_2d
0,df_AESI,0.973451,0.943363


In [68]:
(stock_data_dfs['df_AESI']['Adj Close'] / stock_data_dfs['df_AESI']['Adj Close'].shift(2)).iloc[2]

0.9433627637175842

In [111]:
# Define the columns
columns = ['symbol', 'optimal_day', 'highest_quantile']

# Create an empty list to store data rows
result = pd.DataFrame()

for symbol, df in stock_data_dfs.items():
    
    temp_result = pd.DataFrame({'symbol':[symbol]})
    # Calculate growth rates
    for i in range(1, 31):
        # Ensure that there is an element at index i after the shift
        if i < len(df):  # Check if 'i' is a valid index in the DataFrame after the shift
            growth_value = (df['Adj Close'] / df['Adj Close'].shift(i)).iloc[i]
            temp = pd.DataFrame({
                'symbol': [symbol],
                f'growth_future_' + str(i) + 'd': [growth_value]
            })  # Using a consistent index [0] for easy concatenation
            temp_result = pd.merge(temp_result, temp, how='left', on='symbol')
        else:
            break  # Break the loop if 'i' is out of bounds
    result = pd.concat([result, temp_result], axis=0)

In [112]:
result

,symbol,growth_future_1d,growth_future_2d,growth_future_3d,growth_future_4d,growth_future_5d,growth_future_6d,growth_future_7d,growth_future_8d,growth_future_9d,...,growth_future_21d,growth_future_22d,growth_future_23d,growth_future_24d,growth_future_25d,growth_future_26d,growth_future_27d,growth_future_28d,growth_future_29d,growth_future_30d
0,df_IROH,1.000999,1.000500,1.000999,1.000000,1.000999,1.000000,0.997003,0.997003,0.997003,...,0.997702,0.997702,0.997502,0.997902,0.996004,0.997003,0.997502,0.997003,0.998002,0.998002
0,df_LGCB,0.811224,0.877551,0.795918,0.844898,0.833673,0.859184,0.928571,0.872449,0.806122,...,0.785714,0.790306,0.910204,0.862245,0.831633,0.816327,0.785714,0.795918,0.892857,0.994898
0,df_ZKH,1.000000,1.000000,1.011613,1.008387,1.003871,1.029032,1.025806,1.040000,1.052258,...,1.079355,1.038710,1.045161,1.084516,1.194839,1.228387,1.106452,1.145161,1.169032,1.100645
0,df_BAYA,1.000999,1.003996,1.004995,1.004995,1.004995,1.006893,1.006194,1.005994,1.005994,...,1.011988,1.011988,1.011988,1.011988,1.011988,1.011988,1.011988,1.011988,1.011988,1.011988
0,df_INHD,0.153569,0.133381,0.122206,0.113194,0.108147,0.107426,0.099495,0.090123,0.108508,...,0.077866,0.077145,0.076424,0.076424,0.069430,0.069935,0.081182,0.077866,0.083922,0.081471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,df_PSBD,1.011118,1.004324,1.016677,1.016677,1.008030,1.003088,1.009883,1.002471,1.009883,...,1.016059,1.024089,1.035207,1.006177,1.009265,0.997529,0.990117,1.011118,1.004324,1.004324
0,df_CCTG,1.168831,1.298701,1.267533,1.929870,2.171429,2.529870,2.168831,2.359740,2.751948,...,0.467532,0.381818,0.388312,0.370130,0.358442,0.348052,0.353247,0.393506,0.376623,0.370130
0,df_SYNX,1.011080,1.030471,0.980609,0.850416,0.806094,0.819945,0.798061,0.828255,0.831025,...,0.994460,0.988920,0.941828,0.933518,0.941828,0.977839,0.966759,0.983934,0.969529,0.989474
0,df_SDHC,0.991304,1.010766,1.028986,1.014079,1.002070,1.022360,1.077019,1.033126,1.072464,...,1.117598,1.148240,1.161491,1.157350,1.175569,1.167702,1.186749,1.193789,1.163561,1.171843


In [118]:
# List of growth columns names
growth_columns = ['growth_future_' + str(i) + 'd' for i in range(1, 31)]

# Calculate the 75th quantile for each growth column
quantiles_75th = result[growth_columns].quantile(0.75)

# Identify the day with the highest 75th quantile
print('optimal_day:', quantiles_75th.idxmax())
print('highest_quantile:', quantiles_75th.max())


optimal_day: growth_future_28d
highest_quantile: 1.039141956220915


# Q3

# Q4